In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def gaussian(mu, sig):
    return lambda x: (
        1.0 / (np.sqrt(2.0 * np.pi) * sig) * np.exp(-np.power((x - mu) / sig, 2.0) / 2)
    )

def func(x, bias):
    m = 2
    
    return (x*m) + bias


n_datapoints = 1000

z_range = 1000
z_vals = np.random.uniform(-z_range, z_range, size=n_datapoints)

f1_gauss = gaussian(500, 200)
f2_gauss = gaussian(0, 700)

# f1 = z_vals * -0.5 
# f2 = z_vals * 2 

f1 = f1_gauss(z_vals)
f2 = f2_gauss(z_vals)


plt.scatter(z_vals, f1, label='f1')
plt.scatter(z_vals, f2, label='f2')
plt.ylabel('Feature val')
plt.xlabel('Z position')
plt.legend()
plt.show()


X = np.stack((f1, f2), axis=1)
y = z_vals

In [ ]:
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder


groups = np.zeros((n_datapoints*2, 1))
groups[0:n_datapoints] = 1
groups[n_datapoints:] = 2
groups = groups.astype(str)

onehot_groups = OneHotEncoder().fit_transform(groups)

X2 = X.copy()
X = X + np.random.normal(0, 1e-5, size=X.shape)
X2 = X2 + np.random.normal(0, 1e-5, size=X2.shape)

offset2 = -500
X = np.concatenate((X, X2), axis=0)
y = np.concatenate((y, y+offset2), axis=0)

sns.scatterplot(y, X[:, 0], hue=groups.squeeze())
sns.scatterplot(y, X[:, 1], hue=groups.squeeze())
plt.xlabel('Z position')
plt.ylabel('Feature value')
plt.legend()
plt.show()


In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder().fit(groups)

In [ ]:
SEED = 42

In [ ]:
from sklearn.model_selection import train_test_split
# Without groups informed

idx = np.arange(X.shape[0])

train_idx, test_idx = train_test_split(idx, test_size=0.2, random_state=SEED, stratify=groups)
X_train = X[train_idx]
y_train = y[train_idx]
groups_train = groups[train_idx]

X_test = X[test_idx]
y_test = y[test_idx]
groups_test = groups[test_idx]

In [ ]:
print(X_train.shape, y_train.shape, groups_train.shape)

In [ ]:
groups_train_onehot = encoder.transform(groups_train)
groups_test_onehot = encoder.transform(groups_test)

print(groups_test_onehot.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

out_scaler = StandardScaler()
y_train_scaled = out_scaler.fit_transform(y_train[:, np.newaxis])
y_test_scaled = out_scaler.transform(y_test[:, np.newaxis])

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import optimizers
from tqdm.keras import TqdmCallback
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

feat_input = Input((X_train.shape[1],))

x = feat_input
for n in [25, 25, 25, 1]:
    x = Dense(n, activation='relu' if n!=1 else 'linear')(x)

group_input = Input((groups_train_onehot.shape[1],))
group_layer = Dense(1, use_bias=False)

group_x = group_layer(group_input)

x += group_x

model = Model(inputs=(feat_input, group_input), outputs=x)
print(model.summary())


batch_size = 2000
epochs = 5000
lr = 0.0001

model.compile(loss='mean_squared_error', optimizer=optimizers.AdamW(learning_rate=lr), metrics=['mean_absolute_error'])

callbacks = [
    ReduceLROnPlateau(monitor='val_mean_absolute_error', factor=0.1,
                      patience=25, verbose=True, mode='min', min_delta=1e-5, min_lr=1e-6,),
    TqdmCallback(verbose=0),
    EarlyStopping(monitor='val_mean_absolute_error', patience=50,
                  verbose=False, min_delta=1e-5, restore_best_weights=True),
]
history = model.fit((X_train_scaled, groups_train_onehot), y_train_scaled, validation_data=((X_test_scaled, groups_test_onehot), y_test_scaled), epochs=epochs, verbose=False, callbacks=callbacks)

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [5, 5]
fig, ax1 = plt.subplots()
plt.yscale('log')
ax1.plot(history.history['mean_absolute_error'], label='mse')
ax1.plot(history.history['val_mean_absolute_error'], label='val_mse')
# ax1.set_ylim([0, 500])
ax1.legend(loc=1)
ax2 = ax1.twinx()
ax2.plot(history.history['lr'], label='lr', color='red')
ax2.legend(loc=0)
plt.show()

In [ ]:
print(X_test_scaled.shape)
print(groups_test_onehot.shape)
# tmp = np.zeros(groups_test_onehot.shape)
pred = model.predict((X_test_scaled, groups_test_onehot.toarray()))
y_pred = out_scaler.inverse_transform(pred).squeeze()
print(mean_absolute_error(y_test, y_pred))

print(y_test.shape)
print(y_pred.shape)
print(groups_test.shape)
sns.scatterplot(y_test, y_pred, hue=groups_test.squeeze())
# sns.lineplot([y.min(), y.max()], [y.min(), y.max()])
plt.xlabel('True val')
plt.ylabel('Pred val')
plt.show()

# sns.scatterplot(y_train, X_train[:, 0], hue=X_train[:, 2].astype(str))
# plt.ylabel('Input (feature 1)')
# plt.xlabel('Target')
# plt.show()

# sns.scatterplot(y_train, X_train[:, 1], hue=X_train[:, 2].astype(str))
# plt.ylabel('Input (feature 2)')
# plt.xlabel('Target')
# plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# With groups informed
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

model = MLPRegressor(hidden_layer_sizes=(25 ,25, 25))

scaler = StandardScaler()

X_train[:, [0,1]] = scaler.fit_transform(X_train[:, [0,1]])
X_test[:, [0,1]] = scaler.transform(X_test[:, [0,1]])

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(mean_absolute_error(y_test, y_pred))

sns.scatterplot(y_test, y_pred, hue=X_test[:, 2].astype(str))
sns.lineplot([y.min(), y.max()], [y.min(), y.max()])
plt.xlabel('True val')
plt.ylabel('Pred val')
plt.show()

sns.scatterplot(y_train, X_train[:, 0], hue=X_train[:, 2].astype(str))
plt.ylabel('Input (feature 1)')
plt.xlabel('Target')
plt.show()

sns.scatterplot(y_train, X_train[:, 1], hue=X_train[:, 2].astype(str))
plt.ylabel('Input (feature 2)')
plt.xlabel('Target')
plt.show()

In [ ]:
# Inference without groups
z_range = 1000
z_vals = np.random.uniform(-z_range, z_range, size=n_datapoints)

f1_gauss = gaussian(500, 200)
f2_gauss = gaussian(0, 700)

# f1 = z_vals * -0.5 
# f2 = z_vals * 2 

f1 = f1_gauss(z_vals)
f2 = f2_gauss(z_vals)


plt.scatter(z_vals, f1, label='f1')
plt.scatter(z_vals, f2, label='f2')
plt.ylabel('Feature val')
plt.xlabel('Z position')
plt.legend()
plt.show()


test2_X = np.stack((f1, f2), axis=1)
test2_y = z_vals

test2_X = np.concatenate((test2_X, np.zeros(test2_X.shape)), axis=1)
test2_X[:, 3] = 1

test2_X[:, [0, 1]] = scaler.transform(test2_X[:, [0, 1]])

y_pred = model.predict(test2_X)
plt.scatter(test2_y, y_pred)

plt.plot([-2000, 2000], [-2000, 2000])
plt.show()



In [ ]:
model.coefs_[0]